In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from datetime import timedelta
from datetime import datetime

# Small example demo

In [ ]:
data = {
    'timestamp': [datetime(2023, 1, 1, 12, 0, 0),
                  datetime(2023, 1, 8, 14, 0, 0),
                  datetime(2023, 1, 10, 10, 0, 0),
                  datetime(2023, 1, 16, 8, 0, 0)],
    'user_id': [1, 2, 2, 3],
    'bot': [0, 0, 0, 0],
    'page_id': [1, 1, 1, 1],
}

tmp = pd.DataFrame(data)
tmp = tmp[tmp['bot'] != 1]
tmp['time_diff'] = tmp.groupby('page_id')['timestamp'].diff()


In [ ]:
tmp = tmp.reset_index().drop("index", axis=1).drop("bot", axis=1)

In [ ]:
tmp

,timestamp,user_id,page_id,time_diff
0,2023-01-01 12:00:00,1,1,NaT
1,2023-01-08 14:00:00,2,1,7 days 02:00:00
2,2023-01-10 10:00:00,2,1,1 days 20:00:00
3,2023-01-16 08:00:00,3,1,5 days 22:00:00


In [ ]:
def label_threads(df_page):
    # Create a binary mask indicating when a new thread should start
    mask = (df_page['time_diff'] > timedelta(days=3)).astype(int)
    # Use cumsum to assign unique thread labels
    df_page['thread'] = mask.cumsum().astype(int)

    return df_page

# Apply the function to each group defined by 'page_id'
df_labeled_threads = tmp.groupby('page_id').apply(label_threads)

# Display the result
print(df_labeled_threads)

            timestamp  user_id  page_id       time_diff  thread
0 2023-01-01 12:00:00        1        1             NaT       0
1 2023-01-08 14:00:00        2        1 7 days 02:00:00       1
2 2023-01-10 10:00:00        2        1 1 days 20:00:00       1
3 2023-01-16 08:00:00        3        1 5 days 22:00:00       2


<ipython-input-99-58b3dd9e2343>:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_labeled_threads = tmp.groupby('page_id').apply(label_threads)


# Aggregate more years data

In the robustness check, we see there might not be enough data left for us to do matching and regression anaysis, here are going to aggregate 2 years of data instead of only 2015.

Year 2014-2015 will be aggregated.

Note: I've tried aggregating 3 years of data, but later computation run out of RAM memory. Need resources for this.

In [ ]:
years = range(2014, 2016)

# Initialize an empty DataFrame to store the aggregated data
aggregated_df = pd.DataFrame()

# Loop through each year and concatenate the DataFrames
for year in years:
    print(year)
    file_path = f'/content/drive/MyDrive/capstone/comments_article_{year}/comments_article_{year}_with_predictions.tsv'

    # Read the DataFrame for the current year
    df = pd.read_csv(file_path, delimiter='\t')

    # Concatenate the current year's DataFrame to the aggregated DataFrame
    aggregated_df = pd.concat([aggregated_df, df], ignore_index=True)


2014
2015


In [ ]:
aggregated_df.count()

rev_id            2333307
comment           2333307
raw_comment       2333307
timestamp         2333306
page_id           2333306
page_title        2333301
user_id           2057976
user_text         2333274
bot               2333306
admin             2333306
predicted_prob    2333307
is_attack         2333307
dtype: int64

In [ ]:
aggregated_df["page_id"].nunique()

521856

Three years contain a total of
- 2333307 comments
- 521856 pages

# Data filtering and stats

Again, I have aggregated data from year 2014 and 2015.

Data preprocessing steps:
1. I am dropping all bot comments
2. I'm saving down all comments that appear within 72 hrs time window comparing to their previous or next. (ordered by timestamp)

We are left with
- 122491 pages
- 1518168 comments
- 3991 toxic comments
- 2411 toxic pages

In [ ]:
df = aggregated_df

In [ ]:
df = df[df['bot'] != 1]
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort the DataFrame by 'page_id' and 'timestamp'
df.sort_values(['page_id', 'timestamp'], inplace=True)

<ipython-input-7-59af1aed226d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = pd.to_datetime(df['timestamp'])
<ipython-input-7-59af1aed226d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(['page_id', 'timestamp'], inplace=True)


In [ ]:
df['time_diff'] = df.groupby('page_id')['timestamp'].diff()

occurrences_within_72_hours = df[((df['time_diff'] <= timedelta(days=3)) | (df['time_diff'].shift(-1) <= timedelta(days=3)))]

# Count occurrences within 24 hours for each page
count_within_72_hours = occurrences_within_72_hours.groupby('page_id').size().reset_index(name='page_total_comments')

<ipython-input-8-da8c9c806157>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.groupby('page_id')['timestamp'].diff()


In [ ]:
count_within_72_hours.count()

page_id                122491
page_total_comments    122491
dtype: int64

In [ ]:
count_within_72_hours.describe()

,page_id,page_total_comments
count,1.224910e+05,122491.000000
mean,1.932925e+07,12.394119
std,1.731565e+07,97.255164
min,1.280000e+02,2.000000
25%,3.361810e+06,2.000000
50%,1.249476e+07,3.000000
75%,3.881260e+07,8.000000
max,4.906968e+07,20004.000000


In [ ]:
filtered_df = pd.merge(occurrences_within_72_hours, count_within_72_hours, on='page_id')

In [ ]:
filtered_df.count()

rev_id                 1518168
comment                1518168
raw_comment            1518168
timestamp              1518168
page_id                1518168
page_title             1518165
user_id                1360428
user_text              1518146
bot                    1518168
admin                  1518168
predicted_prob         1518168
is_attack              1518168
time_diff              1427074
page_total_comments    1518168
dtype: int64

In [ ]:
filtered_df["is_attack"].sum()

3991

In [ ]:
num_page_attacks = filtered_df[filtered_df["is_attack"] == 1]["page_id"].nunique()
print(num_page_attacks)

2411


# Convert to thread structure

In [ ]:
filtered_df.sort_values(['page_id', 'timestamp'], inplace=True)

In [ ]:
filtered_df = filtered_df.reset_index().drop("index", axis=1).drop("bot", axis=1)

In [ ]:
filtered_df.columns

Index(['rev_id', 'comment', 'raw_comment', 'timestamp', 'page_id',
       'page_title', 'user_id', 'user_text', 'admin', 'predicted_prob',
       'is_attack', 'time_diff', 'page_total_comments'],
      dtype='object')

In [ ]:
def label_threads(df_page):
    # Create a binary mask indicating when a new thread should start
    mask = (df_page['time_diff'] > timedelta(days=3)).astype(int)
    # Use cumsum to assign unique thread labels
    df_page['thread'] = mask.cumsum().astype(int)

    return df_page

# Apply the function to each group defined by 'page_id'
df_labeled_threads = filtered_df.groupby('page_id').apply(label_threads)

<ipython-input-18-27fc2ccdc4cb>:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_labeled_threads = filtered_df.groupby('page_id').apply(label_threads)


In [ ]:
df_labeled_threads

,rev_id,comment,raw_comment,timestamp,page_id,page_title,user_id,user_text,admin,predicted_prob,is_attack,time_diff,page_total_comments,thread
0,626337159,NEWLINENEWLINE== Movie profitable? ==NEWLINENE...,NEWLINENEWLINE== Movie profitable? ==NEWLINENE...,2014-09-20 12:51:08+00:00,128.0,Atlas Shrugged,436481.0,Prebys,0.0,0.003365,0,218 days 04:16:41,2,1
1,626483998,NEWLINE: Added information about the DVD and B...,NEWLINE: Added information about the DVD and B...,2014-09-21 14:49:43+00:00,128.0,Atlas Shrugged,436481.0,Prebys,0.0,0.022284,0,1 days 01:58:35,2,1
2,595906120,"""NEWLINENEWLINENo, the """"wields"""" is out of th...","""NEWLINENEWLINENo, the """"wields"""" is out of th...",2014-02-17 18:03:26+00:00,692.0,Archaeology,18030372.0,Sɛvɪnti faɪv,0.0,0.032340,0,NaT,5,0
3,595906406,NEWLINENEWLINEWhat's the quote's source? The I...,NEWLINENEWLINEWhat's the quote's source? The I...,2014-02-17 18:06:01+00:00,692.0,Archaeology,18030372.0,Sɛvɪnti faɪv,0.0,0.014016,0,0 days 00:02:35,5,0
4,595910241,NEWLINENEWLINEI'm going to add an actual NPOV ...,NEWLINENEWLINEI'm going to add an actual NPOV ...,2014-02-17 18:35:49+00:00,692.0,Archaeology,18030372.0,Sɛvɪnti faɪv,0.0,0.039590,0,0 days 00:29:48,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518163,692417505,"""NEWLINE:::Update - it turns out that the link...","""NEWLINE:::'''Update''' - it turns out that th...",2015-11-25 15:12:36+00:00,49047223.0,Ashley Reed,21006650.0,Gronk Oz,0.0,0.011618,0,0 days 00:25:31,4,0
1518164,635104747,Has this thing's impact date been ruled out?,* Has this thing's impact date been ruled out?,2014-11-23 14:29:25+00:00,49064567.0,(415029) 2011 UL21,23234112.0,Giga Spaceman,0.0,0.057377,0,NaT,2,0
1518165,635161722,*Edit* The 2029 and all other impact dates ha...,*Edit* The 2029 and all other impact dates ha...,2014-11-23 23:12:55+00:00,49064567.0,(415029) 2011 UL21,23234112.0,Giga Spaceman,0.0,0.037700,0,0 days 08:43:30,2,0
1518166,599713303,}}NEWLINENEWLINE{{reqphoto|in=Guadalajara|of=T...,}}NEWLINE{{WikiProject United States}}NEWLINE{...,2014-03-15 12:16:05+00:00,49069675.0,American School Foundation of Guadalajara,15708.0,WhisperToMe,0.0,0.037354,0,NaT,2,0


In [ ]:
max_thread_stats = df_labeled_threads.groupby('page_id')['thread'].max()
max_thread_stats.describe()

count    122491.000000
mean          1.163359
std           2.775876
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max          70.000000
Name: thread, dtype: float64

Most of the pages only have one thread.

# Filter out pages with only one thread

In [ ]:
labeled_threads_filtered = df_labeled_threads.groupby('page_id').filter(lambda x: x['thread'].nunique() > 1)

In [ ]:
df_labeled_threads_filtered = labeled_threads_filtered.reset_index().drop("index", axis=1)

In [ ]:
df_labeled_threads_filtered

,rev_id,comment,raw_comment,timestamp,page_id,page_title,user_id,user_text,admin,predicted_prob,is_attack,time_diff,page_total_comments,thread
0,595906120,"""NEWLINENEWLINENo, the """"wields"""" is out of th...","""NEWLINENEWLINENo, the """"wields"""" is out of th...",2014-02-17 18:03:26+00:00,692.0,Archaeology,18030372.0,Sɛvɪnti faɪv,0.0,0.032340,0,NaT,5,0
1,595906406,NEWLINENEWLINEWhat's the quote's source? The I...,NEWLINENEWLINEWhat's the quote's source? The I...,2014-02-17 18:06:01+00:00,692.0,Archaeology,18030372.0,Sɛvɪnti faɪv,0.0,0.014016,0,0 days 00:02:35,5,0
2,595910241,NEWLINENEWLINEI'm going to add an actual NPOV ...,NEWLINENEWLINEI'm going to add an actual NPOV ...,2014-02-17 18:35:49+00:00,692.0,Archaeology,18030372.0,Sɛvɪnti faɪv,0.0,0.039590,0,0 days 00:29:48,5,0
3,670508080,NEWLINENEWLINE: Why do you say it is irrelevan...,NEWLINENEWLINE: Why do you say it is irrelevan...,2015-07-08 12:16:48+00:00,692.0,Archaeology,83701.0,Kdammers,0.0,0.004660,0,152 days 02:38:18,5,1
4,670509159,"""NEWLINENEWLINE== Science and Technology of Ar...","""NEWLINENEWLINE== Science and Technology of Ar...",2015-07-08 12:26:41+00:00,692.0,Archaeology,83701.0,Kdammers,0.0,0.002637,0,0 days 00:09:53,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171523,697328536,"""NEWLINE:thanks for clarifying . Harry ""","""NEWLINE:thanks for clarifying [[User:MrArmstr...",2015-12-29 20:10:13+00:00,48901009.0,The Penultimate Curiosity,4548415.0,Harry the Dirty Dog,0.0,0.008296,0,0 days 01:48:50,10,1
1171524,696704402,GA ReviewNEWLINENEWLINE:This review is transcl...,==GA Review==NEWLINE{{Good article tools}}NEWL...,2015-12-25 02:44:11+00:00,48906777.0,Italian ironclad Formidabile/GA1,8731061.0,AustralianRupert,0.0,0.003763,0,NaT,4,0
1171525,696707193,"""NEWLINENEWLINE; CommentsNEWLINEOverall, I bel...","""NEWLINENEWLINE; CommentsNEWLINEOverall, I bel...",2015-12-25 03:22:41+00:00,48906777.0,Italian ironclad Formidabile/GA1,8731061.0,AustralianRupert,0.0,0.001797,0,0 days 00:38:30,4,0
1171526,697134325,NEWLINE**In all likelihood it was published be...,NEWLINE**In all likelihood it was published be...,2015-12-28 13:31:13+00:00,48906777.0,Italian ironclad Formidabile/GA1,1879095.0,Parsecboy,0.0,0.003275,0,3 days 10:08:32,4,1


# Stats

Two years of data, 2014 and 2015.

Already filtered out pages with only one thread, so each page is guaranteed to have more than one threads.

We are left with
- 38002 pages
- 1171528 comments
- 3132 toxic comments
- 1835 toxic pages

Garantees:
1. Each thread has at least two comments
2. Each Page has at least two threads.

In [ ]:
max_thread_stats = df_labeled_threads_filtered.groupby('page_id')['thread'].max()
max_thread_stats.describe()

count    38002.000000
mean         3.248040
std          4.260053
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         70.000000
Name: thread, dtype: float64

In [ ]:
df_labeled_threads_filtered["page_id"].nunique()

38002

In [ ]:
df_labeled_threads_filtered[df_labeled_threads_filtered["is_attack"] == 1]["page_id"].nunique()

1835

In [ ]:
df_labeled_threads_filtered["is_attack"].sum()

3132

Saving down as csv file...

In [ ]:
save_path = f"/content/drive/MyDrive/capstone/pages_with_at_least_two_threads_2014_2015.tsv"
df_labeled_threads_filtered.to_csv(save_path, sep='\t', index=False)

To get number of threads stats as well

In [8]:
save_path = f"/content/drive/MyDrive/capstone/thread_labeled_2001_to_2015/pages_with_at_least_two_threads_2014_2015.tsv"
df_labeled_threads_filtered = pd.read_csv(save_path, sep='\t')

In [9]:
df_labeled_threads_filtered['thread_id'] = df_labeled_threads_filtered.apply(lambda row: (row['page_id'], row['thread']), axis=1)

In [10]:
df_labeled_threads_filtered["thread_id"].nunique()

149106

In [11]:
df_labeled_threads_filtered[df_labeled_threads_filtered["is_attack"] == 1]["thread_id"].nunique()

2281

# Aggregate and process full data

In [ ]:
year = 2006

file_path = f'/content/drive/MyDrive/capstone/comments_article_{year}/comments_article_{year}_10_16_with_predictions.tsv'
df = pd.read_csv(file_path, delimiter='\t')
file_path = f'/content/drive/MyDrive/capstone/comments_article_{year}/comments_article_{year}_0_9_with_predictions.tsv'
df2 = pd.read_csv(file_path, delimiter='\t')
df = pd.concat([df, df2], ignore_index=True)

df = df[df['bot'] != 1]
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.sort_values(['page_id', 'timestamp'], inplace=True)

df['time_diff'] = df.groupby('page_id')['timestamp'].diff()
occurrences_within_72_hours = df[((df['time_diff'] <= timedelta(days=3)) | (df['time_diff'].shift(-1) <= timedelta(days=3)))]

# Count occurrences within 24 hours for each page
count_within_72_hours = occurrences_within_72_hours.groupby('page_id').size().reset_index(name='page_total_comments')
filtered_df = pd.merge(occurrences_within_72_hours, count_within_72_hours, on='page_id')
filtered_df.sort_values(['page_id', 'timestamp'], inplace=True)
filtered_df = filtered_df.reset_index().drop("index", axis=1).drop("bot", axis=1)

def label_threads(df_page):
    # Create a binary mask indicating when a new thread should start
    mask = (df_page['time_diff'] > timedelta(days=3)).astype(int)
    # Use cumsum to assign unique thread labels
    df_page['thread'] = mask.cumsum().astype(int)

    return df_page

# Apply the function to each group defined by 'page_id'
df_labeled_threads = filtered_df.groupby('page_id').apply(label_threads)

labeled_threads_filtered = df_labeled_threads.groupby('page_id').filter(lambda x: x['thread'].nunique() > 1)
df_labeled_threads_filtered = labeled_threads_filtered.reset_index().drop("index", axis=1)

<ipython-input-3-599932c51a59>:31: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_labeled_threads = filtered_df.groupby('page_id').apply(label_threads)


In [ ]:
len(df_labeled_threads_filtered)

1837655

In [ ]:
max_thread_stats = df_labeled_threads_filtered.groupby('page_id')['thread'].max()
max_thread_stats.describe()

count    56416.000000
mean         4.030381
std          4.309024
min          1.000000
25%          2.000000
50%          2.000000
75%          5.000000
max         39.000000
Name: thread, dtype: float64

In [ ]:
df_labeled_threads_filtered[df_labeled_threads_filtered["is_attack"] == 1]["page_id"].nunique()

5053

In [ ]:
df_labeled_threads_filtered["is_attack"].sum()

10823

In [ ]:
save_path = f"/content/drive/MyDrive/capstone/thread_labeled_2001_to_2015/pages_with_at_least_two_threads_{year}.tsv"
df_labeled_threads_filtered.to_csv(save_path, sep='\t', index=False)

2001:
- 75 pages
- 1064 comments
- 2 toxic comments
- 2 toxic pages

2002:
- 757 pages
- 12698 comments
- 51 toxic comments
- 28 toxic pages

2003:
- 2280 pages
- 40962 comments
- 113 toxic comments
- 87 toxic pages

2004:
- 7746 pages
- 184272 comments
- 685 toxic comments
- 381 toxic pages

2005:
- 23184 pages
- 657417 comments
- 3226 toxic comments
- 1703 toxic pages

2006:
- 56416 pages
- 1837655 comments
- 10823 toxic comments
- 5053 toxic pages

2007:
- 56832 pages
- 1799479 comments
- 12499 toxic comments
- 5329 toxic pages

2008:
- 44658 pages
- 1419630 comments
- 9620 toxic comments
- 4048 toxic pages

2009:
- 38208 pages
- 1168692 comments
- 7453 toxic comments
- 3291 toxic pages

2010:
- 32710 pages
- 988305 comments
- 5003 toxic comments
- 2440 toxic pages

2011:
- 29061 pages
- 857921 comments
- 3006 toxic comments
- 1682 toxic pages

2012:
- 24648 pages
- 691073 comments
- 2299 toxic comments
- 1367 toxic pages

2013:
- 20674 pages
- 572332 comments
- 1699 toxic comments
- 991 toxic pages

2014:
- 18923 pages
- 538263 comments
- 1458 toxic comments
- 886 toxic pages

2015;
- 19620 pages
- 550756 comments
- 1389 toxic comments
- 897 toxic pages

Garantees:
1. Each thread has at least two comments
2. Each Page has at least two threads.